<a href="https://colab.research.google.com/github/danielvilaca/IA24_P07/blob/main/ProjAlter_IA24_P07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto IA24 Grupo 07 (2.0 | Modelo Alternativo)

## Imports



*   cp_model: Para definir e resolver problemas de otimização baseados em restrições.
*   files: Permite fazer o upload de ficheiros no Google Colab.
*   pprint: Usada para imprimir dados em formato mais legível.
*   matplotlib.pyplot: Para a gerar gráficos, como o gráfico de Gantt.
*   re: Usada para trabalhar com expressões regulares no processamento de texto.




In [ ]:
# Instalar a biblioteca OR-Tools
!pip install ortools

# Importar bibliotecas necessárias
from ortools.sat.python import cp_model
from google.colab import files
from pprint import pprint
import matplotlib.pyplot as plt
import re

## Função de Upload do Ficheiro

*    Objetivo: Permite o upload de ficheiros no ambiente do Colab para serem utilizados no processamento de dados.
*    files.upload(): Exibe um seletor para fazer o upload dos ficheiros.
*    fn: Itera sobre os nomes dos ficheiros enviados.

In [2]:
# Função para fazer o upload do file no Colab
def upload_file():
    uploaded = files.upload()
    for fn in uploaded.keys():
        print(f'Ficheiro "{fn}" enviado com sucesso!')
    return list(uploaded.keys())[0]  # Retorna o nome do primeiro ficheiro enviado

## Função para Gerar o Gráfico de Gantt

*    Objetivo: Visualizar as tarefas programadas ao longo do tempo, representando sua ordem e duração.
*    plt.subplots: Cria uma figura e eixos para o gráfico.
*    barh: Gera barras horizontais para representar a duração de cada tarefa.
*    ax.grid: Adiciona uma grelha ao gráfico para facilitar a leitura.
*    set_xticks e set_yticks: Define os intervalos no eixo X (tempo) e Y (tarefas).

In [ ]:
# Função para gerar o gráfico de Gantt
def gerar_gantt(tarefas, start_times, end_times, solver):
    tasks = []
    for tarefa in tarefas:
        tarefa_id = tarefa['id']
        inicio = solver.Value(start_times[tarefa_id])
        fim = solver.Value(end_times[tarefa_id])
        tasks.append((tarefa_id, inicio, fim))

    tasks.sort(key=lambda x: x[1])  # Ordenar pelo início

    fig, ax = plt.subplots(figsize=(15, 9))
    for i, (tarefa_id, inicio, fim) in enumerate(tasks):
        ax.barh(i, fim - inicio, left=inicio, color='skyblue', edgecolor='black', height=0.8)

    ax.set_xticks(range(0, 50, 1))
    ax.set_yticks(range(len(tasks)))
    ax.set_yticklabels([f"Tarefa {t[0]}" for t in tasks])
    ax.invert_yaxis()
    ax.grid(axis='y', linestyle='--', color='gray', alpha=0.5)
    ax.grid(axis='x', linestyle='--', color='gray', alpha=0.5)
    ax.set_xlabel('Tempo')
    ax.set_ylabel('Tarefas')
    ax.set_title('Gráfico de Gantt - Sequência das Tarefas')
    plt.tight_layout()
    plt.show()

## Função para Processar o Ficheiro de Entrada (Parsing)

*    Objetivo: Lê e interpreta o ficheiro de entrada, separando as informações de tarefas, precedências e recursos.
*    max_resources: Dicionário para armazenar os recursos disponíveis.
*    tarefas: Lista de dicionários, cada um representando uma tarefa com id, duração e recursos necessários.
*    precedencies: Dicionário que mapeia cada tarefa às suas predecessoras.
Secções do Ficheiro:

    *    Recursos: Define a quantidade máxima disponível de cada recurso.
    *    Duração e Recursos: Associa as tarefas às suas durações e requisitos de recursos.
    *    Relações de Precedência: Especifica quais tarefas devem preceder outras.

In [ ]:
# Função para processar o ficheiro de entrada
def parse_text_file(file_path):
    tarefas = []
    precedencies = {}
    max_resources = {}

    with open(file_path, 'r') as f:
        lines = f.readlines()

    resource_section = False
    task_section = False
    precedence_section = False

    for line in lines:
        line = line.strip()

        # Identifica início de secções
        if line.startswith('#Resource availability'):
            resource_section = True
        elif line.startswith('#Duration and resources'):
            resource_section = False
            task_section = True
        elif line.startswith('#Precedence relations'):
            task_section = False
            precedence_section = True

        # Processa a secção de recursos
        if resource_section and re.match(r'^\s*R\d+', line):
            parts = line.split()
            resource_type = parts[0]
            resource_qty = int(parts[1])
            max_resources[resource_type] = resource_qty

        # Processa a secção das tarefas
        elif task_section and re.match(r'^\s*\d+', line):
            parts = line.split()
            task_id = int(parts[0])
            duration = int(parts[2])
            task_resources = {f'R{i-2}': int(parts[i]) for i in range(3, len(parts))}
            tarefas.append({'id': task_id, 'duration': duration, 'resources': task_resources})

        # Processa a secção das precedências
        elif precedence_section and re.match(r'^\s*\d+', line):
            parts = line.split()
            task_id = int(parts[0])
            successors = list(map(int, parts[3:]))
            precedencies[task_id] = successors

    return tarefas, precedencies, max_resources

## Função Principal para Modelar, Resolver e Exibir Resultados

*    Objetivo: Cria e resolve o modelo de otimização utilizando OR-Tools, exibindo os resultados e o gráfico de Gantt.
*    start_times e end_times: Variáveis que representam os tempos de início e fim de cada tarefa.
*    intervals: Lista de intervalos associados a cada tarefa no modelo.
*    model.Add: Adiciona restrições de precedência e uso de recursos ao modelo.
*    solver: Resolve o problema usando cp_model.CpSolver.

In [ ]:
# Função principal para criar o modelo, resolver e exibir os resultados
def schedule_tasks(file_path):
    # Lê o ficheiro e processa as estruturas de dados
    tarefas, precedencies, max_resources = parse_text_file(file_path)

    print("\nPrecedências:")
    pprint(precedencies)
    print("\nRecursos Máximos:")
    pprint(max_resources)
    print("\n")

    # Criar modelo do OR-Tools
    model = cp_model.CpModel()

    # Definir variáveis e intervalos
    start_times = {}
    end_times = {}
    intervals = []
    soma_duracoes = sum(tarefa['duration'] for tarefa in tarefas)

    for tarefa in tarefas:
        tarefa_id = tarefa['id']
        duration = tarefa['duration']
        start_times[tarefa_id] = model.NewIntVar(0, soma_duracoes, f"start_{tarefa_id}")
        end_times[tarefa_id] = model.NewIntVar(0, soma_duracoes, f"end_{tarefa_id}")
        interval = model.NewIntervalVar(start_times[tarefa_id], duration, end_times[tarefa_id], f"interval_{tarefa_id}")
        intervals.append(interval)

    # Adicionar restrições de precedência
    for tarefa_id, dependencies in precedencies.items():
        for dep in dependencies:
            model.Add(end_times[tarefa_id] <= start_times[dep])

    # Adicionar restrições dos recursos
    for resource, max_qty in max_resources.items():
        demands = [tarefa['resources'].get(resource, 0) for tarefa in tarefas]
        model.AddCumulative(intervals, demands, max_qty)

    # Definir objetivo
    duracao_max_projeto = model.NewIntVar(0, 100000000000, "duracao_max_projeto")
    model.AddMaxEquality(duracao_max_projeto, [end_times[tarefa['id']] for tarefa in tarefas])
    model.Minimize(duracao_max_projeto)

    # Resolver o modelo
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Exibir resultados
    if status == cp_model.OPTIMAL:
        print("Sequência ótima encontrada:\n")
    elif status == cp_model.FEASIBLE:
        print("Sequência viável encontrada:\n")

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"{'Tarefa':<10} {'Início':>20} {'Término':>10}")
        print("-" * 50)
        for tarefa in tarefas:
            tarefa_id = tarefa['id']
            inicio = solver.Value(start_times[tarefa_id])
            fim = solver.Value(end_times[tarefa_id])
            print(f"Tarefa {tarefa_id:<10} {inicio:>10} {fim:>10}")

        gerar_gantt(tarefas, start_times, end_times, solver)
        print(f"\nDuração do Projeto: {solver.Value(duracao_max_projeto)}\n")
    else:
        print("Nenhuma solução encontrada!")
        print("Status do solver:", solver.StatusName(status))


## Execução Principal

*   upload_file: Faz o upload do ficheiro de entrada para o Colab.
*   schedule_tasks: Chama a função principal para processar o ficheiro e resolver o problema.

In [ ]:
# Execução principal
file_path = upload_file()  # Faz o upload do ficheiro no Colab
schedule_tasks(file_path)  # Processa o ficheiro e gera a solução

# Referências

*   *OR-Tools Documentation*

    OR-Tools Documentation. Disponível em:
    https://developers.google.com/optimization.

*   *Google Colab Documentation*

    Google Colab Help Center. Disponível em:
    https://colab.research.google.com.

*   *CP-SAT Solver Guide*
    
    Constraint Programming Solver (CP-SAT) Overview. Disponível em:
    https://developers.google.com/optimization/cp.

*   *Pawel Lichocki - OR-tools*

    Disponível em:
    https://www.youtube.com/watch?v=AJ6LeiMe_PQ&ab_channel=MixedIntegerProgramming

*   *Matplotlib Documentation*

    Matplotlib: Visualization with Python. Disponível em:
    https://matplotlib.org/stable/contents.html.

*   *Python Official Documentation*

    The Python Standard Library. Disponível em:
    https://docs.python.org/3/library/.


*   *Scheduling Optimization Reference*

    Pinedo, M. L. Scheduling: Theory, Algorithms, and Systems. 5th Edition, Springer, 2016.

*   *Regular Expressions Reference*

    Friedl, J. E. F. Mastering Regular Expressions. 3rd Edition, O'Reilly Media, 2006.

*   *Gantt Chart Basics*

    Wilson, R. Project Management for Engineers and Developers. Elsevier, 2014.